In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
replace resized/9733.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm
import collections

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.models import Model
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Input, Reshape, GlobalAveragePooling2D
from keras import regularizers
from keras.regularizers import l2
from keras.applications.resnet_v2 import ResNet50V2
from keras.layers import Input
from keras.models import Model
from keras import backend as K
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping, ModelCheckpoint, LearningRateScheduler

Using TensorFlow backend.


In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [0]:
class_weights = {}
cl_names = ["gender_output", "image_quality_output", "age_output", "weight_output", "bag_output", "footwear_output", "emotion_output", "pose_output"]
cols = list(df.columns)
cols.pop()
i = 0
for col in cols:
    a = df[col].value_counts(normalize=True)
    od = collections.OrderedDict(sorted(dict(a).items()))
    temp = {}
    ix = 0
    for d in od:
        temp[ix] = od[d]
        ix +=1
    class_weights[cl_names[i]] = temp
    i +=1

print(class_weights)

{'gender_output': {0: 0.43741251013040594, 1: 0.5625874898695941}, 'image_quality_output': {0: 0.5532306785530097, 1: 0.16503352243424446, 2: 0.28173579901274587}, 'age_output': {0: 0.18374714506741324, 1: 0.39865910263022175, 2: 0.25307595962572754, 3: 0.10977676269063583, 4: 0.054741029986001624}, 'weight_output': {0: 0.6356737640904737, 1: 0.06564503057540706, 2: 0.23546747218743094, 3: 0.06321373314668828}, 'bag_output': {0: 0.33912915346644074, 1: 0.09732557282840934, 2: 0.56354527370515}, 'footwear_output': {0: 0.37044131732115226, 1: 0.18470492890296913, 2: 0.4448537537758786}, 'emotion_output': {0: 0.11051351948721727, 1: 0.11854416856995506, 2: 0.7117070654976793, 3: 0.05923524644514846}, 'pose_output': {0: 0.16260222500552568, 1: 0.6176232225742282, 2: 0.21977455242024607}}


In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
import keras
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None, im_size=224):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation
        self.im_size = im_size

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        ims = [cv2.imread(item["image_path"]) for _, item in items.iterrows()]
        image = np.stack([cv2.resize(im, (self.im_size, self.im_size)) for im in ims])
        image = image.astype('float32') / 255
        images_mean = np.mean(image, axis=0)
        image -= images_mean

        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        if self.augmentation is not None:
          image = self.augmentation.flow(image, shuffle=False, batch_size=self.batch_size).next()

        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.2,random_state=31)
train_df.shape, val_df.shape

((10858, 28), (2715, 28))

In [0]:
def augmentor(images):
		'Apply data augmentation'
		sometimes = lambda aug: iaa.Sometimes(0.5, aug)
		seq = iaa.Sequential(
				[
				# apply the following augmenters to most images
				iaa.Fliplr(0.3),  # horizontally flip 50% of all images
				sometimes(iaa.Affine(
					scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
					# scale images to 80-120% of their size, individually per axis
					translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
					# translate by -20 to +20 percent (per axis)
					rotate=(-10, 10),  # rotate by -45 to +45 degrees
					shear=(-5, 5),  # shear by -16 to +16 degrees
					order=[0, 1],
					# use nearest neighbour or bilinear interpolation (fast)
					cval=(0, 255),  # if mode is constant, use a cval between 0 and 255
					mode=ia.ALL
					# use any of scikit-image's warping modes (see 2nd image from the top for examples)
				)),
				# execute 0 to 5 of the following (less important) augmenters per image
				# don't execute all of them, as that would often be way too strong
				iaa.SomeOf((0, 5),
				           [sometimes(iaa.Superpixels(p_replace=(0, 1.0),
						                                     n_segments=(20, 200))),
					           # convert images into their superpixel representation
					           iaa.OneOf([
							           iaa.GaussianBlur((0, 1.0)),
							           # blur images with a sigma between 0 and 3.0
							           iaa.AverageBlur(k=(3, 5)),
							           # blur image using local means with kernel sizes between 2 and 7
							           iaa.MedianBlur(k=(3, 5)),
							           # blur image using local medians with kernel sizes between 2 and 7
					           ]),
					           iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)),
					           # sharpen images
					           iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
					           # emboss images
					           # search either for all edges or for directed edges,
					           # blend the result with the original image using a blobby mask
					           iaa.SimplexNoiseAlpha(iaa.OneOf([
							           iaa.EdgeDetect(alpha=(0.5, 1.0)),
							           iaa.DirectedEdgeDetect(alpha=(0.5, 1.0),
							                                  direction=(0.0, 1.0)),
					           ])),
					           iaa.AdditiveGaussianNoise(loc=0,
					                                     scale=(0.0, 0.01 * 255),
					                                     per_channel=0.5),
					           # add gaussian noise to images
					           
					           
					           iaa.Add((-2, 2), per_channel=0.5),
					           # change brightness of images (by -10 to 10 of original value)
					           iaa.AddToHueAndSaturation((-1, 1)),
					           # change hue and saturation
					           # either change the brightness of the whole image (sometimes
					           # per channel) or change the brightness of subareas
					           iaa.OneOf([
							           iaa.Multiply((0.9, 1.1), per_channel=0.5),
							           iaa.FrequencyNoiseAlpha(
									           exponent=(-1, 0),
									           first=iaa.Multiply((0.9, 1.1),
									                              per_channel=True),
									           second=iaa.ContrastNormalization(
											           (0.9, 1.1))
							           )
					           ]),
					           sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5),
					                                               sigma=0.25)),
					           # move pixels locally around (with random strengths)
					           sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))),
					           # sometimes move parts of the image around
					           sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
				           ],
				           random_order=True
				           )
				],
				random_order=True
		)
		return seq.augment_images(images)
  

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):

  """
  p : the probability that random erasing is performed
  s_l, s_h : minimum / maximum proportion of erased area against input image
  r_1, r_2 : minimum / maximum aspect ratio of erased area
  v_l, v_h : minimum / maximum value for erased area
  pixel_level : pixel-level randomization for erased area
  """

  def eraser(input_img):
    img_h, img_w, img_c = input_img.shape
    p_1 = np.random.rand()

    if p_1 > p:
        return input_img

    while True:
        s = np.random.uniform(s_l, s_h) * img_h * img_w
        r = np.random.uniform(r_1, r_2)
        w = int(np.sqrt(s / r))
        h = int(np.sqrt(s * r))
        left = np.random.randint(0, img_w)
        top = np.random.randint(0, img_h)

        if left + w <= img_w and top + h <= img_h:
            break

    if pixel_level:
        c = np.random.uniform(v_l, v_h, (h, w, img_c))
    else:
        c = np.random.uniform(v_l, v_h)

    input_img[top:top + h, left:left + w, :] = c

    return input_img

  return eraser


aug = ImageDataGenerator(
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=2,
        # randomly shift images horizontally
        width_shift_range=0.08,
        # randomly shift images vertically
        height_shift_range=0.05,
        # set range for random shear
        shear_range=0.07,
        # set range for random zoom
        zoom_range=0.05,
        # set range for random channel shifts
        channel_shift_range=0.07,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        preprocessing_function=get_random_eraser(p=0.08,v_l=0, v_h=0.3, s_l=0.008, s_h=0.05, pixel_level=False)
				)

In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32,augmentation=aug)
valid_gen = PersonDataGenerator(val_df, batch_size=32, shuffle=False)

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
model = ResNet50V2(weights= None, include_top=False, input_tensor=Input(shape=(224, 224, 3)))

neck = model.output
neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)


def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128,activation='relu')(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))

model = Model(inputs=[model.input], outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion])













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [0]:
class CyclicLR(keras.callbacks.Callback):
    
    def __init__(self,base_lr, max_lr, step_size, base_m, max_m, cyclical_momentum):
 
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.base_m = base_m
        self.max_m = max_m
        self.cyclical_momentum = cyclical_momentum
        self.step_size = step_size
        
        self.clr_iterations = 0.
        self.cm_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}
        
    def clr(self):
        
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        
        if cycle == 2:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)          
            return self.base_lr-(self.base_lr-self.base_lr/100)*np.maximum(0,(1-x))
        
        else:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0,(1-x))
    
    def cm(self):
        
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        
        if cycle == 2:
            
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1) 
            return self.max_m
        
        else:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
            return self.max_m - (self.max_m-self.base_m)*np.maximum(0,(1-x))
        
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())
            
        if self.cyclical_momentum == True:
            if self.clr_iterations == 0:
                K.set_value(self.model.optimizer.momentum, self.cm())
            else:
                K.set_value(self.model.optimizer.momentum, self.cm())
            
            
    def on_batch_begin(self, batch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        
        if self.cyclical_momentum == True:
            self.history.setdefault('momentum', []).append(K.get_value(self.model.optimizer.momentum))

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())
        
        if self.cyclical_momentum == True:
            K.set_value(self.model.optimizer.momentum, self.cm())

In [0]:
losses = {
	"gender_output": "binary_crossentropy",
	"image_quality_output": "categorical_crossentropy",
	"age_output": "categorical_crossentropy",
	"weight_output": "categorical_crossentropy",
  "bag_output": "categorical_crossentropy",
  "footwear_output": "categorical_crossentropy",
  "pose_output": "categorical_crossentropy",
  "emotion_output": "categorical_crossentropy"
}

loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0, "weight_output": 1.0, "bag_output": 1.0, "footwear_output": 1.0, "pose_output": 1.0, "emotion_output": 1.0}
opt = SGD(lr=0.001, momentum=0.9)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy",
    #loss=losses, 
    #loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [0]:
epochs = 100
max_lr = 0.0001
base_lr = max_lr/100
max_m = 0.98
base_m = 0.85

augment = True
cycles = 8

iterations = round(len(train_df)/32*epochs)
iterations = list(range(0,iterations+1))
step_size = len(iterations)/(cycles)


clr =  CyclicLR(base_lr=base_lr,
                max_lr=max_lr,
                step_size=step_size,
                max_m=max_m,
                base_m=base_m,
                cyclical_momentum=True)



import os

if not os.path.exists('gdrive/My Drive/saved_models'):
    os.mkdir('gdrive/My Drive/saved_models')
LOGS_DIR = 'gdrive/My Drive/saved_models'

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger(LOGS_DIR +'/'+'log.csv')
filepath=os.path.join(LOGS_DIR+os.sep+'model-{epoch:03d}-{val_loss:.4f}.hdf5')
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [16]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=epochs,
    verbose=1,
    #class_weight= class_weights,
    #callbacks=[lr_reducer, early_stopper, csv_logger, checkpoint, LearningRateScheduler(scheduler, verbose=1)],
    callbacks=[checkpoint,clr]
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
339/339 [==============================] - 207s 612ms/step - loss: 8.5421 - gender_output_loss: 0.7273 - image_quality_output_loss: 1.0463 - age_output_loss: 1.5673 - weight_output_loss: 1.0768 - bag_output_loss: 1.0121 - footwear_output_loss: 1.0857 - pose_output_loss: 0.9867 - emotion_output_loss: 1.0399 - gender_output_acc: 0.5260 - image_quality_output_acc: 0.5084 - age_output_acc: 0.3273 - weight_output_acc: 0.5982 - bag_output_acc: 0.5086 - footwear_output_acc: 0.4575 - pose_output_acc: 0.5871 - emotion_output_acc: 0.6650 - val_loss: 8.3247 - val_gender_output_loss: 0.7489 - val_image_quality_output_loss: 1.0361 - val_age_output_loss: 1.4986 - val_weight_output_loss: 1.0491 - val_bag_output_loss: 0.9682 - val_footwear_output_loss: 1.0765 - val_pose_output_loss: 0.9762 - val_emotion_output_loss: 0.9711 - val_gender_output_acc: 0.5454 - val_image_quality_output_acc: 0.5193 - 

Process ForkPoolWorker-450:
Process ForkPoolWorker-452:
Process ForkPoolWorker-448:
Process ForkPoolWorker-447:
Process ForkPoolWorker-446:
Process ForkPoolWorker-445:
Process ForkPoolWorker-449:
Process ForkPoolWorker-455:
Process ForkPoolWorker-453:
Process ForkPoolWorker-456:
Process ForkPoolWorker-454:
Process ForkPoolWorker-451:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  Fi

KeyboardInterrupt: ignored

In [0]:
from keras.models import load_model
model = load_model('gdrive/My Drive/saved_models/model-030-7.6081.hdf5')

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=epochs,
    verbose=1,
    initial_epoch=37,
    #class_weight= class_weights,
    #callbacks=[lr_reducer, early_stopper, csv_logger, checkpoint, LearningRateScheduler(scheduler, verbose=1)],
    callbacks=[checkpoint,clr]
)

Epoch 38/100
339/339 [==============================] - 213s 628ms/step - loss: 7.6420 - gender_output_loss: 0.6483 - image_quality_output_loss: 0.9752 - age_output_loss: 1.4056 - weight_output_loss: 0.9787 - bag_output_loss: 0.9030 - footwear_output_loss: 0.9080 - pose_output_loss: 0.9109 - emotion_output_loss: 0.9123 - gender_output_acc: 0.6074 - image_quality_output_acc: 0.5528 - age_output_acc: 0.3968 - weight_output_acc: 0.6346 - bag_output_acc: 0.5713 - footwear_output_acc: 0.5810 - pose_output_acc: 0.6182 - emotion_output_acc: 0.7100 - val_loss: 7.6091 - val_gender_output_loss: 0.6466 - val_image_quality_output_loss: 0.9724 - val_age_output_loss: 1.4089 - val_weight_output_loss: 0.9772 - val_bag_output_loss: 0.9021 - val_footwear_output_loss: 0.9072 - val_pose_output_loss: 0.9079 - val_emotion_output_loss: 0.8868 - val_gender_output_acc: 0.6101 - val_image_quality_output_acc: 0.5543 - val_age_output_acc: 0.4003 - val_weight_output_acc: 0.6365 - val_bag_output_acc: 0.5592 - val_f